In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import geopy.distance as gd
import os
from pandas import ExcelWriter
import numpy as np
import gmplot
import matplotlib.pyplot as plt 
from functools import reduce
import sqlite3 
pd.options.display.max_rows = 40
pd.options.display.max_columns = 100

plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

# Python Analysis Functions

In [ ]:
def clean_df(df, col = 'crash_date', min_year = 2019, lat_col = 'latitude', lon_col = 'longitude', geo = True) :
    '''
    Performs various cleaning to a dataframe
    
    inputs:
    df (pandas dataframe): dataframe with at minimum crash_date, latitude, and longitude columns
    col (string): column to convert to datetime
    min_year (int): year to filter dataframe >=
    lat_col (string): name of latitude column
    lon_col (string): name of longitude column
    geo (Bool): if True, drop rows which have nan coordinates
    
    output:
    cleaned df
    '''
    #Convert all columns to lowercase and replace spaces with _
    df.columns= df.columns.str.lower().str.replace(' ', '_')
    
    #Convert date column to datetime
    df[col] = pd.to_datetime(df[col])
    
    #Filter by min_year
    df = df.loc[(df[col].dt.year >= min_year)]
    df.reset_index(inplace = True, drop = True)
    
    if geo :
        #Drop any coordinates with na
        df.dropna(subset=[lat_col, lon_col], inplace = True)
#         df = df.loc[0:500] #Uncomment after finishing code
        
    return df

def clean_df_simple(df, lat_col, lon_col) :
    #Convert all columns to lowercase and replace spaces with _
    df.columns= df.columns.str.lower().str.replace(' ', '_')

def clean_serv_df(serv_df, svi_df) :
    '''
    Cleans and merges the service df and the servicibility_index df
    to be used for Tableau visualization
    
    Inputs: 
    serv_df (Pandas dataframe): service dataframe
    svi_df (Pandas dataframe): servicibility_index dataframe
    
    Outputs:
    pivot_df (Pandas dataframe): merged dataframe
    
    '''
    
    #Calculate Total route length / total stop_count = Bus Stop Spacing (ft.)
    serv_df = serv_df.groupby(['Route Type', 'Route']).sum().reset_index()
    serv_df['Bus Stop Spacing (ft.)'] = serv_df['Route Length (mi)'] / serv_df['stop_count'] * 5280
    
    #Join with svi_df
    merged_df = pd.merge(serv_df, svi_df, on=['Route'], how = 'outer')
    merged_df.sort_values(by = ['Route'], inplace = True)
#     display(merged_df.head())
    
    #Reformat the dataframe
    pivot_df = pd.melt(merged_df, id_vars = 'Route', var_name = 'Type', value_name = 'Value')
    
    #Order by Route
    pivot_df.sort_values(by = ['Route'], inplace = True)
    pivot_df.reset_index(inplace = True)
#     display(pivot_df.head(30))
    return pivot_df
    
def merge_shapefile(data_df, shape_file, crs_string = 'epsg:4269') :
    '''
    data_df : dataframe with some columns including 'longitude' and 'latitude'
    shape_file: .shp shape file gpd
    
    returns - merged_df which is dataframe consisting of the data from data_df, joined with the shape_file
    '''
    
    #Get lat and long of the points in the data_df
    geometry = [Point(xy) for xy in zip(data_df.longitude, data_df.latitude)]
    crs = {'init' :crs_string}
    #Create geodataframe from the lats and longs
    gdf = gpd.GeoDataFrame(data_df, crs=crs, geometry=geometry)
#     display(gdf.head())
    #Merge the geodataframe with the shape_file
    merged_df = gpd.sjoin(gdf, shape_file, how='left', op='within')
    
    return merged_df

def calc_dist(df, start_lat_col = 'latitude', start_lon_col = 'longitude',\
              end_lat = 41.7383652, end_lon = -87.7313064, new_col = 'dist') :
    '''
    input:
        df: dataframe with latitude and longitude columns
        start_lat_col: name of latitude column in df (string)
        start_lon_col: name of longitude column in df (string)
        end_lat: end latitude (i.e. 41.7383652) (float)
        end_lon: end longitude (i.e. -87.7313064) (float)
        end_lat, end_lon : coordinate to calculate distance from lat/longs given in df
        new_col: new column name to store distances in, i.e. 'dist' (string)
        
    output:
        returns nothing, but edits df in place to include new distance column
        
        '''
    df[new_col] = df.apply(\
                          (lambda row: gd.distance((row[start_lat_col], row[start_lon_col]),\
                                                   (end_lat, end_lon)).miles), axis=1)
def filter_by_dist(df, start_lat_col = 'Start Lat', start_lon_col = 'Start Long',\
              end_lat = 41.7383652, end_lon = -87.7313064, new_col = 'dist', dist = 0.25) :

    '''
    input:
        df: dataframe with latitude and longitude columns
        new_col: new column name to store distances in, i.e. 'dist' (string)
        dist: filtering distance in miles (i.e. 0.25)
        
    output:
        returns the df filtered by the specified distance
        
        '''
    cond1 = (df[new_col] <= dist)
    df = df.loc[cond1]
    return df

def add_census_tracts(df) :
    '''
    Adds census tract populations of the same census tract
    df: census_population data with columns 'Census Tract' and 'Population'
    
    returns df
    '''
    new_df = df.groupby(['Census Tract']).sum()
    new_df.reset_index(inplace = True)
    return new_df
    
def create_lookup_table(df, col1, col2) :
    '''
    Creates lookup dictionary with col1 as the key and col2 as the value
    df: df with col1 and col2
    col1: string name of column (assumes col1 type should be float)
    col2: string name of another column
    
    returns dictionary
    '''
    comm_df = df.loc[:, [col1, col2]]
    comm_df[col1] = comm_df[col1].astype(float)
    comm_df.set_index(col1, inplace = True)
#     display(comm_df.loc[17031700302])
#     comm_df.drop_duplicates(inplace = True) Was dropping too much for some reason...
#     display(comm_df.loc[17031700302])
    comm_dict = comm_df.to_dict()
    comm_dict = comm_dict[col2]
#     print(comm_dict[17031700302.0])
    return comm_dict

def gmap_plotter(lats, longs, path, zoom = 15) :
    '''
    Plots a series of lat and long points on google maps
    
    Inputs:
    lats, longs (pandas series): series values of lats and longs
    path (string): path to save gmplot
    zoom (int): zoom level for map
    
    Outputs:
    function returns nothing, but a HTML google maps plot is saved to specified path
    '''
    
    #zoom level 0 - widest zoom, >> - narrowest zoom
    
    #lats are series
    lat_start = lats.iloc[int(round(lats.shape[0]/2))]
    
    #longs are series
    long_start = longs.iloc[int(round(longs.shape[0]/2))]
    
    #starting point of map
    gmap = gmplot.GoogleMapPlotter(lat_start, long_start, zoom)
    
    #Plot all points
    gmap.scatter(lats, longs,'#FF0000', \
                              size = 3 , marker = False) 
    
    #Big first two points (green)
    gmap.scatter(lats.iloc[0:2], longs.iloc[0:2], '#00ff0d', marker = False, size = 100)
    
    #Big last two points (red)
    gmap.scatter(lats.iloc[-2:], longs.iloc[-2:], '#FF0000', marker = False, size = 100)
    
    #Plot the entire path
    #gmap.plot(lats, longs, 'cornflowerblue', edge_width = 4)
    
    #Plot to html
    gmap.draw(path) 

# Plotting and Visualization

In [ ]:
def bar_plot_by_val(df, val, col1, col2):
    '''
    Inputs:
        df: original dataframe to filter from
        col1, val: set col1 = val and filter df
        col2: bar chart x axis value
    
    Output:
        return: bar plot
        
    '''
    #Filter df col by val
#     temp_df = df.groupby(['student_id']).max().reset_index()
    temp_df = df.loc[(df[col1] == val), [col2]]
    temp_df[col2].value_counts(ascending=True).sort_index(ascending = True)\
    .plot(kind = 'bar')
    plt.xlabel(col2)
    plt.ylabel('number of _')
    plt.title(val)
    plt.show()

# SQL

In [ ]:
with pysqlite3.connect(path) as connection:
    cursor = connection.cursor()
    
def sql_func(query, cols) :
    '''
    Inputs:
        query: SQL query string
               Example:
               
                    SELECT r.id, s.deg, s.group,
                    s.organization, c.course_id, c.course_title,
                    c.credits, 

                    SUM(c.credits) OVER (PARTITION BY r.id) 
                    ORDER BY r.term_date) as runnning_total_credits,

                    SUM(c.credits) OVER (PARTITION BY r.id) 
                    as total_credits, r.term_date,

                    MIN(r.term_date) OVER (PARTITION BY r.id)
                    as min_term_date, 

                    MAX(r.term_date) OVER (PARTITION BY r.id)
                    as max_term_date

                    FROM (registration r 
                    LEFT JOIN student s
                    ON r.id = s.id) as q1
                    LEFT JOIN course c 
                    ON q1.course_id = c.course_id

                    ORDER BY s.organization, r.id, r.term_date
               
        cols: list of columns to label dataframe
    Output:
        dataframe of query result
        
        '''
    cursor.execute(query)
    df = cursor.fetchall()
    df = pd.DataFrame(data = df, columns = cols)
    return df